# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 09 de abril de 2024, 09:00 hrs.

**Nombre alumno:** Sebastian Navea Aguirre


# Apache Hive

## Instalación de ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
exec(open('/content/drive/MyDrive/BigDataSw/hadoop_colab_installer.py').read())

Active services:
2432 JobHistoryServer
2146 ResourceManager
2485 Jps
2293 DataNode
2377 NodeManager
2220 NameNode



## Descarga de dataset

In [ ]:
!cp /content/drive/MyDrive/BigDataSw/lastfm-dataset-1K.tar.gz .
!tar xzf lastfm-dataset-1K.tar.gz

## Actividad 0

In [ ]:
# Inserte su código para copia de datos en HDFS y creación de tablas en metastore

#primero, creacion de directorio destinatario para archivo 1 userid-profile
!hdfs dfs -mkdir user_profile
#creacion directorio destinatario para archivo 2  userid-timestamp-artid
!hdfs dfs -mkdir user_timestamp

#copiar datos user_profile a carpeta recien creada
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv user_profile
#copiar datos userid_timestamp a carpeta recien creada
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv user_timestamp



In [ ]:
#creacion base de datos, user, pass, nombre bd
!mysql -u root --password=password -e "CREATE DATABASE db_lastfm;"
#Uso de base de datos recien creada
!mysql -u root --password=password db_lastfm

mysql: [Warning] Using a password on the command line interface can be insecure.
mysql: [Warning] Using a password on the command line interface can be insecure.
Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 14
Server version: 8.0.36-0ubuntu0.22.04.1 (Ubuntu)

Copyright (c) 2000, 2024, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> show databases;
+--------------------+
| Database           |
+--------------------+
| db_lastfm          |
| information_schema |
| mysql              |
| performance_schema |
| sys                |
| testdb             |
+--------------------+
6 rows in set (0.00 sec)

mysql> show tables;
Empty set (0.00 sec)

mysql> use db_lastfm;
Database changed
mysql> exit;
Bye


In [ ]:
#Creacion de tabla externa dentro del metastore para datos alojados en HDFS
#en tabla user_timestamp, se cambio el nombre de columna timestamp a ts, ya que timestamp es una
#reservada en hive
%%writefile create_tables.sql

CREATE EXTERNAL TABLE user_profile (
    userid STRING,
    gender STRING,
    age INT,
    country STRING,
    signup STRING
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://localhost:9000/user/root/user_profile';

CREATE EXTERNAL TABLE user_timestamp (
    userid STRING,
    ts STRING,
    musicbrainz_artist_id STRING,
    artist_name STRING,
    musicbrainz_track_id STRING,
    track_name STRING
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION 'hdfs://localhost:9000/user/root/user_timestamp';

Writing create_tables.sql


In [ ]:
!hive -f create_tables.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
OK
Time taken: 14.654 seconds
OK
Time taken: 0.334 seconds


## Actividad 1

In [ ]:
# Inserte su código
%%writefile select_top10_artist.sql
select artist_name, COUNT(*) AS play_count FROM user_timestamp
GROUP BY artist_name ORDER BY play_count DESC
LIMIT 10;

Writing select_top10_artist.sql


In [ ]:
!hive -f select_top10_artist.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20240408205136_201a5105-249f-44da-bce8-96b90c323906
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 10
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.

Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular dele ranking top-10 ?

El artista mas popular es Radiohead, con 115209 reproduccion, y el menos popuular es Elliott Smith, con 50278 reproducciones.

## Actividad 2

In [ ]:
# Inserte su código
#desarrollo de select considerando el artista mas escuchado, obtener la
#distribucion de sus oyentes segun el genero
%%writefile select_most_played_by_gender.sql

select p.gender, COUNT(*) as listener_count
FROM user_profile p JOIN user_timestamp t ON p.userid = t.userid
WHERE t.artist_name = 'Radiohead' AND p.gender IS NOT NULL
GROUP BY p.gender;

Writing select_most_played_by_gender.sql


In [ ]:
!hive -f select_most_played_by_gender.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20240408205857_39932d0b-e150-4db0-8674-398b0a46dbe1
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?

La cantidad de mujeres que han escuchado alguna cancion del artista polular son 43748.

La cantidad de hombres que han esuchado alguna cancion del artista popular son de 63784


## Actividad 3

In [ ]:
# Inserte su código
#desarrollo de select considerando el artista mas escuchado, obtener
#la distribucion de los auditores por su edad, ordenado ascendente por edad

%%writefile select_most_played_by_age.sql

select p.age, COUNT(*) as listener_count
FROM user_profile p JOIN user_timestamp t ON p.userid = t.userid
WHERE t.artist_name = 'Radiohead' AND p.age IS NOT NULL
GROUP BY p.age
ORDER BY p.age ASC;

Writing select_most_played_by_age.sql


In [ ]:
!hive -f select_most_played_by_age.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20240408211854_ce63cfec-5631-40f8-bde9-9b13f3898059
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?

Dentro de los usuarios de 35 años, 1543 de ellos han escuchado al artista mas popular.